In [1]:
# Imports
import os
import torch
import torchvision
import torch.nn as nn  
import torch.nn.functional as F  
import torchvision.datasets as datasets  
import torchvision.transforms as transforms  
from torch.utils.data import DataLoader
import numpy as np  
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

PR curves

In [ ]:
writer = SummaryWriter(
    f"runs/classes"
)
# 1. gets the probability predictions in a test_size x num_classes Tensor
# 2. gets the preditions in a test_size Tensor
# takes ~10 seconds to run
class_probs = []
class_label = []
with torch.no_grad():
    for inputs, labels in dataloaders["val"]:
        outputs = model(inputs)
        class_probs_batch = [F.softmax(output, dim=0) for output in outputs]    # N, num_class

        class_probs.append(class_probs_batch)   # num_batch, N, num_class
        class_label.append(labels)              # num_batch, N, 1

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])   # size, num_class
test_label = torch.cat(class_label)                                     # Size, 1

# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_label, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_label)

In [ ]:
# num_epochs = 1
# batch_sizes = [2, 8, 16]
# learning_rates = [0.001, 0.01]
# classes = image_dataset["train"].classes
# experiment = "mat"

# for batch_size in batch_sizes:
#     for learning_rate in learning_rates:
#         step = 0
#         # Initialize network
#         model.to(device)
#         model.train()
#         train_loader = DataLoader(
#             dataset=train_dataset, batch_size=batch_size, shuffle=True
#         )
#         criterion = nn.CrossEntropyLoss()
#         optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
#         writer = SummaryWriter(
#             f"runs/{experiment}/MiniBatchSize {batch_size} LR {learning_rate}"
#         )

#         # Visualize model in TensorBoard
#         images, _ = next(iter(train_loader))
#         writer.add_graph(model, images.to(device))
#         writer.close()

#         for epoch in range(num_epochs):
#             losses = []
#             accuracies = []

#             for batch_idx, (data, targets) in enumerate(train_loader):
#                 # Get data to cuda if possible
#                 data = data.to(device=device)
#                 targets = targets.to(device=device)

#                 # forward
#                 scores = model(data)
#                 loss = criterion(scores, targets)
#                 losses.append(loss.item())

#                 # backward
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()

#                 # Calculate 'running' training accuracy
#                 features = data.reshape(data.shape[0], -1)
#                 img_grid = torchvision.utils.make_grid(data)
#                 _, predictions = scores.max(1)
#                 num_correct = (predictions == targets).sum()
#                 running_train_acc = float(num_correct) / float(data.shape[0])
#                 accuracies.append(running_train_acc)

#                 # Plot things to tensorboard
#                 class_labels = [classes[label] for label in predictions]
#                 writer.add_image("images", img_grid)
#                 writer.add_histogram("fc", model.classifier.fc.weight)
#                 writer.add_scalar("Training loss", loss, global_step=step)
#                 writer.add_scalar(
#                     "Training Accuracy", running_train_acc, global_step=step
#                 )

#                 if batch_idx == 230:
#                     writer.add_embedding(
#                         features,
#                         metadata=class_labels,
#                         label_img=data,
#                         global_step=batch_idx,
#                     )
#                 step += 1

#             writer.add_hparams(
#                 {"lr": learning_rate, "batch size": batch_size},
#                 {
#                     "accuracy": sum(accuracies) / len(accuracies),
#                     "loss": sum(losses) / len(losses),
#                 },
#             )